In [ ]:
# Run this once. agar errors aaye to message bhejo.
!pip install folium pandas rapidfuzz gTTS SpeechRecognition pydub --quiet
# pydub requires ffmpeg; Colab mein usually preinstalled. Agar error aaye: uncomment next line.
!apt-get install -y ffmpeg > /dev/null 2>&1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from IPython.display import display, HTML

csv_path = "/content/college_locations.csv"
df = pd.read_csv(csv_path)

print("Loaded CSV OK — rows:", len(df))
display(df.head(30))  # shows top rows
# Ensure columns are exactly as expected
print("\nColumns:", list(df.columns))


Loaded CSV OK — rows: 26


,Location,Details
0,EB201,Located in 1st Floor engineering block
1,EB202,Located in 1st Floor engineering block
2,EB203,Located in 1st Floor engineering block
3,EB204,Located in 1st Floor engineering block
4,EB205,Located in 1st Floor engineering block
5,EB206,Located in 1st Floor engineering block
6,EB207,Located in 1st Floor engineering block
7,EB208,Located in 1st Floor engineering block
8,EB301,Located in 2nd Floor engineering block
9,EB302,Located in 2nd Floor engineering block



Columns: ['Location', 'Details']


In [ ]:
# Standardize column names to known names (but do not add/remove rows)
df = df.rename(columns=lambda c: c.strip())  # trim spaces
# map typical names to our used names
col_map = {}
if "Location" in df.columns:
    col_map["Location"] = "place_name"
if "Details" in df.columns:
    col_map["Details"] = "location_details"

df = df.rename(columns=col_map)
# if category missing, create blank column (we won't auto-fill)
if "category" not in df.columns:
    df["category"] = ""

print("Final columns:", list(df.columns))
display(df.head(20))


Final columns: ['place_name', 'location_details', 'category']


,place_name,location_details,category
0,EB201,Located in 1st Floor engineering block,
1,EB202,Located in 1st Floor engineering block,
2,EB203,Located in 1st Floor engineering block,
3,EB204,Located in 1st Floor engineering block,
4,EB205,Located in 1st Floor engineering block,
5,EB206,Located in 1st Floor engineering block,
6,EB207,Located in 1st Floor engineering block,
7,EB208,Located in 1st Floor engineering block,
8,EB301,Located in 2nd Floor engineering block,
9,EB302,Located in 2nd Floor engineering block,


In [ ]:
from rapidfuzz import process, fuzz

# Prepare choices from your CSV (exactly as in file)
choices = df["place_name"].astype(str).tolist()

def fuzzy_search(query, limit=3, score_cutoff=50):
    # Returns top matches as DataFrame (only from your CSV)
    if not isinstance(query, str) or query.strip()=="":
        return pd.DataFrame()
    q = query.strip()
    results = process.extract(q, choices, scorer=fuzz.WRatio, limit=limit)
    # results: list of (match, score, index)
    matched = []
    for match, score, idx in results:
        if score >= score_cutoff:
            row = df.iloc[idx].copy()
            row["_match_score"] = score
            matched.append(row)
    if not matched:
        return pd.DataFrame()
    return pd.DataFrame(matched)

# Quick test (text input)
print(fuzzy_search("eb204"))
print(fuzzy_search("Auditorium"))


  place_name                         location_details category  _match_score
3      EB204  Located in 1st Floor engineering block                    60.0
               place_name                       location_details category  \
14  Auditorium IGSM Block  Auditorium in IGSM Block ground floor            

    _match_score  
14          90.0  


In [ ]:
from gtts import gTTS
from IPython.display import Audio
import os
import tempfile

def speak_text(text):
    # saves temp mp3 and plays inline
    tts = gTTS(text=text, lang="en")  # english TTS; if want Hindi use lang="hi"
    tmp = tempfile.NamedTemporaryFile(suffix=".mp3", delete=False)
    tts.save(tmp.name)
    display(Audio(tmp.name, autoplay=True))
    # file remains for playback; can remove if you want
    return tmp.name

# Example: search by typed text and speak top 1 result
q = input("Type the location you want to search (example: EB204 or Auditorium): ").strip()
res = fuzzy_search(q, limit=1)
if res.empty:
    msg = "Sorry, koi match nahin mila. Try different keywords."
    print(msg)
    speak_text(msg)
else:
    row = res.iloc[0]
    msg = f"Location: {row['place_name']}. Details: {row['location_details']}."
    print(msg)
    speak_text(msg)


Type the location you want to search (example: EB204 or Auditorium): IOS LABS
Location: IOS lab. Details: an apple based lab in admission block building.
